<a href="https://colab.research.google.com/github/viktoruebelhart/Keras_NPL_News/blob/main/word_completer_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We built a Deep Learning model to classify articles into different categories. Now, a new need has emerged: our content portal wants editors and journalists to follow a consistent style when writing texts. To do this, we will create a word completer that follows a specific style, similar to the feature found in text editors.

In [1]:
import pandas as pd
import tensorflow as tf

url = 'https://github.com/allanspadini/curso-tensorflow-proxima-palavra/raw/main/dados/train.zip'

df = pd.read_csv(url, header=None, names=['ClassIndex', 'Título', 'Descrição'])

df['Texto'] = df['Título'] + '' + df['Descrição']